In [ ]:
%matplotlib inline
from pyleecan.Functions.save import save
from pyleecan.Functions.load import load
from pyleecan.Functions.load import load_matlib
from pyleecan.definitions import DATA_DIR
from util.toyota_prius_generator import Toyota_Prius_Generator

from pyleecan.Classes.MachineIPMSM import MachineIPMSM
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.InputCurrent import InputCurrent
import matplotlib.pyplot as plt
from numpy import sqrt
from os import makedirs
from os.path import join
# Load the machine
from util.simulation import *
from util.failures import *
from IPython.display import display, HTML


In [ ]:
# Dossier de sauvegarde des machines
save_dir = "machines_custom6"
makedirs(save_dir, exist_ok=True)

In [ ]:
#  Génération de la machine avec Ntcoil = 60
gen_60 = Toyota_Prius_Generator(Ntcoil=60)
machine_60 = MachineIPMSM(
    name="Toyota_Prius_Ntcoil_60_geom_modif",
    shaft=gen_60.create_shaft(),
    rotor=gen_60.create_rotor(),
    stator=gen_60.create_stator()
)

# Chargement des matériaux
mat_air = load(join(DATA_DIR, "Material", "Air.json"))
mat_fe = load(join(DATA_DIR, "Material", "M400-50A.json"))

# Vérification des matériaux
if machine_60.stator.mat_type is None:
    machine_60.stator.mat_type = mat_fe
if machine_60.rotor.mat_type is None:
    machine_60.rotor.mat_type = mat_fe
if machine_60.frame is not None and machine_60.frame.mat_type is None:
    machine_60.frame.mat_type = mat_air

# Modification de la géométrie : valeurs choisies pour observer des effets mesurables
machine_60.stator.slot.H0 = 0.0018  # hauteur base de l'encoche stator
machine_60.stator.slot.W0 = 0.0021  # largeur d'ouverture stator
machine_60.rotor.hole[0].H0 = 0.012  # profondeur du trou rotor
machine_60.rotor.hole[0].W0 = 0.048  # largeur du trou rotor


save(machine_60, join(save_dir, machine_60.name + ".json"))

print("Machine avec Ntcoil=60 et géométrie modifiée enregistrée avec succès.")

In [ ]:
print(machine_60.rotor.hole[0].magnet_0.mat_type)

In [ ]:
print(machine_60.stator.mat_type.name)
print(machine_60.rotor.mat_type.name)
print(machine_60.frame.mat_type if machine_60.frame else "No frame")

In [ ]:
print(machine_60.shaft.mat_type.name)


In [ ]:
import os
from pyleecan.definitions import DATA_DIR

mat_dir = os.path.join(DATA_DIR, "Material")
print("Fichiers disponibles :", os.listdir(mat_dir))


In [ ]:
# chargement des matériaux
mat_air = load(join(DATA_DIR, "Material", "Air.json"))
mat_cu = load(join(DATA_DIR, "Material", "Copper1.json"))
mat_steel = load(join(DATA_DIR, "Material", "M400-50A.json"))
mat_shaft = load(join(DATA_DIR, "Material", "Steel1.json"))
mat_mag = load(join(DATA_DIR, "Material", "MagnetPrius.json"))

# application à la machine
machine_60.shaft.mat_type = mat_shaft
machine_60.stator.mat_type = mat_steel
machine_60.rotor.mat_type = mat_steel

# on pplique aux aimants et aux régions d’air du rotor
for hole in machine_60.rotor.hole:
    if hole.magnet_0:
        hole.magnet_0.mat_type = mat_mag
    if hasattr(hole, "magnet_1") and hole.magnet_1:
        hole.magnet_1.mat_type = mat_mag


In [ ]:
%matplotlib inline

# Load the machine
IPMSM_A = load("machines_custom6/Toyota_Prius_Ntcoil_60_geom_modif.json")
# In Jupyter notebook, we set is_show_fig=False to skip call to fig.show() to avoid a warning message
# All plot methods return the corresponding matplotlib figure and axis to further edit the resulting plot
fig, ax = IPMSM_A.plot(is_show_fig=False)
plt.show()

In [ ]:
print(IPMSM_A.stator.winding.wind_mat)

In [ ]:
verification  de machine 100....

In [ ]:
%matplotlib inline

# Load the machine
IPMSM_A = load("machines_custom_batch_100_ntcoil/216_machine_correcte_simuler/Toyota_Prius_healthy_3_Ntcoil_10.json")
# In Jupyter notebook, we set is_show_fig=False to skip call to fig.show() to avoid a warning message
# All plot methods return the corresponding matplotlib figure and axis to further edit the resulting plot
fig, ax = IPMSM_A.plot(is_show_fig=False)
plt.show()

In [ ]:
# Chargement des machines
machine_ref = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))
machine_ntcoil = load("machines_custom6/Toyota_Prius_Ntcoil_60_geom_modif.json")

In [ ]:
simu_ref = load_simulation(name="ref", machine=machine_ref, rotor_speed=3000, stop=1, num_steps=4000)
simu_ntcoil = load_simulation(name="ntcoil", machine=machine_ntcoil, rotor_speed=3000, stop=1, num_steps=4000)

In [ ]:
out_ref = simu_ref.run()
out_ntcoil = simu_ntcoil.run()

compare_simulation_results(
    out1=out_ref,
    out2=out_ntcoil,
    legend_list=["Toyota_Prius_Ref", "Toyota_Prius_Ntcoil_60_geom_modif"]
)

In [ ]:
out_ref.save(save_path='machines_custom6/compar_simule', is_folder=True, type_handle_old=2, type_compression=0)